# Traffic visualization using `sumolib` and `edge-data`

In [ ]:
from matplotlib.collections import LineCollection
from matplotlib import pyplot as plt
import matplotlib
from lxml import etree
import subprocess

if "SUMO_HOME" in os.environ:
    tools = os.path.join(os.environ["SUMO_HOME"], "tools")
    sys.path.append(tools)
else:
    sys.exit("Please declare the environment variable 'SUMO_HOME'")

import sumolib
from sumolib.visualization import helpers

plt.rcParams["figure.figsize"] = 30, 20
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

n = "traci-n"
rr = "traci-rr"

# fig, (ax1, ax2) = plt.subplots(2,1)
# plot_heatmap(normal, ax1)
# plot_heatmap(rr, ax2)
# fig.set_figheight(50)

fig, ax = plt.subplots()
plot_edge_data(n, ax)

In [ ]:
def plot_edge_data(scenario, ax, scenario2=None, relative=False):
    edgeData = f"{scenario}/output/edge-data-full-day.xml"
    cmap = matplotlib.cm.get_cmap('winter_r')

    if scenario2 is not None:
        edgeData2 = f"{scenario2}/output/edge-data-full-day.xml"
        edgeDataDiff = f"{os.path.basename(scenario)}-{os.path.basename(scenario2)}-edge-data-diff.xml"
        command = [
            "python", 
            os.path.join(os.environ["SUMO_HOME"], "tools", "output", "edgeDataDiff.py"), 
            edgeData, 
            edgeData2, 
            edgeDataDiff
        ]
        if relative:
            command.append("--relative")
            command.append("--undefined=0")
        subprocess.run(command)
        
        edgeData = edgeDataDiff
        cmap = matplotlib.cm.get_cmap('bwr')

    net = sumolib.net.readNet(f"{scenario}/network/network.net.xml")

    default_color = [0.95 for i in range(3)]
    colors = {}

    default_width = 1
    widths = {}

    for event, edge in etree.iterparse(edgeData, tag="edge"):
        c = float(edge.attrib["entered"])
        colors[edge.attrib["id"]] = c
            
        w = float(edge.attrib["entered"])
        widths[edge.attrib["id"]] = w

    # Normalize colors
    min_color = min(colors.values())
    max_color = max(colors.values())
    max_color_diff = max(abs(min_color), max_color)

    for edge in colors:
        if colors[edge] == 0:
            colors[edge] = default_color
        else:
            if scenario2 is None:
                colors[edge] = cmap(colors[edge] / max_color)
            else:
                colors[edge] = cmap((colors[edge] + max_color_diff) / (max_color_diff * 2))

    # Normalize width
    min_width = min(widths.values())
    max_width = max(widths.values())
    max_width_diff = max(abs(min_width), max_width)
    for edge in widths:
        width = (abs(widths[edge]) / max_width_diff) * 2
        if width > default_width:
            widths[edge] = width
        else:
            widths[edge] = default_width

    shapes = []
    c = []
    w = []
    for e in net.getEdges():
        shapes.append(e.getShape())
        c.append(colors[str(e.getID())])
        w.append(widths[str(e.getID())])


    line_segments = LineCollection(shapes, linewidths=w, colors=c)
    ax.add_collection(line_segments)
    ax.set_xmargin(0.1)
    ax.set_ymargin(0.1)
    ax.autoscale_view(True, True, True)

    
    cbar_min = 1
    cbar_max = max_color_diff

    if scenario2 is not None:
        cbar_min = max_color_diff * -1

    norm = matplotlib.colors.Normalize
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm(vmin=cbar_min,vmax=cbar_max))
    plt.colorbar(sm, ax=ax)